In [1]:
import os
import time
import numpy as np
import soundfile as sf
import scipy
import csv
import scipy.misc
import imageio
import shutil
import sys
import datetime
import random
import collections
import pickle
import math

from random import randint

from keras.backend.tensorflow_backend import set_session
import tensorflow as tf
# set the memory usage
tf_config = tf.ConfigProto()
tf_config.gpu_options.allow_growth = True
set_session(tf.Session(config=tf_config))

from keras.layers import MaxPooling1D, Dropout, AveragePooling2D,LSTM,Reshape, Bidirectional, Add
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Input, BatchNormalization,Conv1D
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping
from keras import backend as K
from keras import losses, optimizers
from keras.layers.pooling import GlobalMaxPooling1D

Using TensorFlow backend.


In [2]:
frames_per_sample = 200

def create_melspec_train(path):
    train_frames=[]
    train_labels=[]
    with open('event_list_devtrain_gunshot.csv', 'rt') as train_file:
        reader = list(csv.reader(train_file, delimiter='\n'))
        
        for row,i in zip(reader,range(5000)):
            fields= str(row).split('\\t')
            
            temp_train_frames =[]
            temp_train_labels = []
                    
            if(len(fields)>1):
                if(fields[1] and fields[2]):
                    file_name = fields[0].replace("[\'", "").replace("\']", "")
                    melspec = imageio.imread(path + str(i) + '.png')
                    print(file_name)
                    event_start_index = int(np.round(float(fields[1])/30 * 1305))
                    event_finish_index = int(np.round(float(fields[2])/30 * 1305))
                    event_length = event_finish_index - event_start_index 
                    print(event_start_index)
                    print(event_finish_index)
                    
                    #calculating sample start and finish indices, randomly
                    sample_start = randint(max(0,event_start_index - (200-event_length)),event_start_index)
                    sample_finish = min(1305,sample_start+frames_per_sample)
                    
                    print('before')
                    print(sample_start)
                    print(sample_finish)
                    sample_length = sample_finish - sample_start 
                    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
                        sample_start = sample_start - (frames_per_sample - sample_length)
                    
                    print('after')
                    print(sample_start)
                    print(sample_finish)
                    
                    temp1 = np.array(melspec)
                    temp_train_frames = temp1[:,sample_start:sample_finish]
                    
                    print(np.array(temp_train_frames).shape)
                    for ind in range(sample_start,event_start_index):
                        temp_train_labels.append(0)
                    for ind in range(event_start_index,event_finish_index):
                        temp_train_labels.append(1)
                    for ind in range(event_finish_index,sample_finish):
                        temp_train_labels.append(0)
                    
                    print('temp_train_labels.shape is: '+str(np.array(temp_train_labels).shape))   
                    temp2 = np.array(temp_train_frames)
                    temp2 = temp2.reshape(128,200)
                    train_frames.append(temp2)
                    
                    temp3 = np.array(temp_train_labels)
                    temp3 = temp3.reshape(200)
                    train_labels.append(temp3)
                    
                    print(np.array(train_frames).shape)
                    print(np.array(train_labels).shape)
            if(i%500==0):
                print('Reading files: '+str(i))
    
    train_file.close()
    train_data = {'train_frames':train_frames,
                 'train_labels':train_labels}
    return train_data

In [3]:
file_path = 'train_5000_gunshot_Prob1_44100_128/'
d={}
d = create_melspec_train(file_path)

mixture_devtrain_gunshot_000_d8de4c6012991d172265358d2f5d39c1.wav
589
632
before
435
635
after
435
635
(128, 200)
temp_train_labels.shape is: (200,)
(1, 128, 200)
(1, 200)
Reading files: 0
mixture_devtrain_gunshot_001_dffc115bd35f642ea2ab4b7d6ae24ce5.wav
589
632
before
558
758
after
558
758
(128, 200)
temp_train_labels.shape is: (200,)
(2, 128, 200)
(2, 200)
mixture_devtrain_gunshot_002_c8c5425d9fa907cd2eb7de04a9d2a7bb.wav
340
423
before
274
474
after
274
474
(128, 200)
temp_train_labels.shape is: (200,)
(3, 128, 200)
(3, 200)
mixture_devtrain_gunshot_003_1ecf97826e009c310ed4c6362409f2f9.wav
439
454
before
424
624
after
424
624
(128, 200)
temp_train_labels.shape is: (200,)
(4, 128, 200)
(4, 200)
mixture_devtrain_gunshot_004_9b09eeb56d0ec8d5f2e8c1221e596e6d.wav
445
464
before
423
623
after
423
623
(128, 200)
temp_train_labels.shape is: (200,)
(5, 128, 200)
(5, 200)
mixture_devtrain_gunshot_005_524992c2fd186606cfabf18050631399.wav
587
605
before
476
676
after
476
676
(128, 200)
temp_trai

mixture_devtrain_gunshot_054_3d889b1b3d96753553cc2948e5268325.wav
386
481
before
378
578
after
378
578
(128, 200)
temp_train_labels.shape is: (200,)
(55, 128, 200)
(55, 200)
mixture_devtrain_gunshot_055_12c1b6342f1487d92d77b71522f8b0f2.wav
1088
1141
before
1015
1215
after
1015
1215
(128, 200)
temp_train_labels.shape is: (200,)
(56, 128, 200)
(56, 200)
mixture_devtrain_gunshot_056_aacfd4e2a1118da54fc431b7964d451e.wav
466
548
before
420
620
after
420
620
(128, 200)
temp_train_labels.shape is: (200,)
(57, 128, 200)
(57, 200)
mixture_devtrain_gunshot_057_7ebaa1296c2bde492bda65da0963b112.wav
830
858
before
807
1007
after
807
1007
(128, 200)
temp_train_labels.shape is: (200,)
(58, 128, 200)
(58, 200)
mixture_devtrain_gunshot_058_e67a7f28e03b8ddd6f9e88e0ffdc4d0a.wav
255
310
before
149
349
after
149
349
(128, 200)
temp_train_labels.shape is: (200,)
(59, 128, 200)
(59, 200)
mixture_devtrain_gunshot_059_4af2187ebb9c43043c3f50fbf3d06e87.wav
799
810
before
653
853
after
653
853
(128, 200)
temp_tra

(126, 128, 200)
(126, 200)
mixture_devtrain_gunshot_126_97e2e5371645257a7083e45b9d675c02.wav
439
502
before
394
594
after
394
594
(128, 200)
temp_train_labels.shape is: (200,)
(127, 128, 200)
(127, 200)
mixture_devtrain_gunshot_127_4fdcf7e3e19a051232b6e9445c9b7427.wav
254
358
before
239
439
after
239
439
(128, 200)
temp_train_labels.shape is: (200,)
(128, 128, 200)
(128, 200)
mixture_devtrain_gunshot_128_2180ce89083d104d83780f13b11881bb.wav
977
1008
before
861
1061
after
861
1061
(128, 200)
temp_train_labels.shape is: (200,)
(129, 128, 200)
(129, 200)
mixture_devtrain_gunshot_129_62ff1c74f6157cd959815db3245f93d6.wav
1130
1227
before
1053
1253
after
1053
1253
(128, 200)
temp_train_labels.shape is: (200,)
(130, 128, 200)
(130, 200)
mixture_devtrain_gunshot_130_813dd96a42912e920b16cba603a71d7e.wav
311
362
before
164
364
after
164
364
(128, 200)
temp_train_labels.shape is: (200,)
(131, 128, 200)
(131, 200)
mixture_devtrain_gunshot_131_42bb22c59553a68214a4f1e8ad6c3622.wav
785
814
before
630

mixture_devtrain_gunshot_196_14424dd7e4543f29b00c5d8d47e0b003.wav
364
398
before
224
424
after
224
424
(128, 200)
temp_train_labels.shape is: (200,)
(197, 128, 200)
(197, 200)
mixture_devtrain_gunshot_197_26d85953b986d8fdb2c5fba9bbfb0681.wav
278
294
before
217
417
after
217
417
(128, 200)
temp_train_labels.shape is: (200,)
(198, 128, 200)
(198, 200)
mixture_devtrain_gunshot_198_83230973b5989841ea38338859b76694.wav
434
451
before
315
515
after
315
515
(128, 200)
temp_train_labels.shape is: (200,)
(199, 128, 200)
(199, 200)
mixture_devtrain_gunshot_199_7dc144a577a3ec05b68ca41fca925904.wav
761
851
before
686
886
after
686
886
(128, 200)
temp_train_labels.shape is: (200,)
(200, 128, 200)
(200, 200)
mixture_devtrain_gunshot_200_48dc5287ccedc23b81d6d4231534a9a2.wav
511
527
before
508
708
after
508
708
(128, 200)
temp_train_labels.shape is: (200,)
(201, 128, 200)
(201, 200)
mixture_devtrain_gunshot_201_305b282ed7d071018f76f0dc6d75081b.wav
1194
1288
before
1092
1292
after
1092
1292
(128, 200)


mixture_devtrain_gunshot_252_eb3cd92bfad0ac114bab94b6760d11ae.wav
520
578
before
480
680
after
480
680
(128, 200)
temp_train_labels.shape is: (200,)
(253, 128, 200)
(253, 200)
mixture_devtrain_gunshot_253_50cc1e5098907f11729d1be217764f83.wav
1085
1208
before
1020
1220
after
1020
1220
(128, 200)
temp_train_labels.shape is: (200,)
(254, 128, 200)
(254, 200)
mixture_devtrain_gunshot_254_bb5d81238f598fdd7e81a3d653b948fd.wav
951
1051
before
946
1146
after
946
1146
(128, 200)
temp_train_labels.shape is: (200,)
(255, 128, 200)
(255, 200)
mixture_devtrain_gunshot_255_5a8748f733a5414f28d0ac58276a4d38.wav
29
119
before
5
205
after
5
205
(128, 200)
temp_train_labels.shape is: (200,)
(256, 128, 200)
(256, 200)
mixture_devtrain_gunshot_256_1bb3d26d2b9aca406c87d4e91e443362.wav
760
779
before
727
927
after
727
927
(128, 200)
temp_train_labels.shape is: (200,)
(257, 128, 200)
(257, 200)
mixture_devtrain_gunshot_257_23d42f98e5694c9c5def3f150cfd1060.wav
254
358
before
190
390
after
190
390
(128, 200)
te

...

(4794, 128, 200)
(4794, 200)
mixture_devtrain_gunshot_4794_805053d8d5661d94f457b6c18610c6f5.wav
436
476
before
349
549
after
349
549
(128, 200)
temp_train_labels.shape is: (200,)
(4795, 128, 200)
(4795, 200)
mixture_devtrain_gunshot_4795_6a342b9241f4f146149860c6dd88e520.wav
1126
1210
before
1056
1256
after
1056
1256
(128, 200)
temp_train_labels.shape is: (200,)
(4796, 128, 200)
(4796, 200)
mixture_devtrain_gunshot_4796_5c3c4caa9efe295476404f763ce3cec3.wav
548
609
before
445
645
after
445
645
(128, 200)
temp_train_labels.shape is: (200,)
(4797, 128, 200)
(4797, 200)
mixture_devtrain_gunshot_4797_6a46b35f422530770d822e13617e015e.wav
411
495
before
393
593
after
393
593
(128, 200)
temp_train_labels.shape is: (200,)
(4798, 128, 200)
(4798, 200)
mixture_devtrain_gunshot_4798_ae316cedf7fd127a11c157940dc8c8fe.wav
815
848
before
760
960
after
760
960
(128, 200)
temp_train_labels.shape is: (200,)
(4799, 128, 200)
(4799, 200)
mixture_devtrain_gunshot_4799_77ff7748b29360b523fea68840368482.wav
192

(4841, 128, 200)
(4841, 200)
mixture_devtrain_gunshot_4841_c28a361a91023e8767a2255820c43e14.wav
72
97
before
7
207
after
7
207
(128, 200)
temp_train_labels.shape is: (200,)
(4842, 128, 200)
(4842, 200)
mixture_devtrain_gunshot_4842_0be68860b38591da59febd5207466523.wav
975
1040
before
891
1091
after
891
1091
(128, 200)
temp_train_labels.shape is: (200,)
(4843, 128, 200)
(4843, 200)
mixture_devtrain_gunshot_4843_b55219ab393289bafd67ef24873ce27f.wav
184
257
before
167
367
after
167
367
(128, 200)
temp_train_labels.shape is: (200,)
(4844, 128, 200)
(4844, 200)
mixture_devtrain_gunshot_4844_906844304ec0029dd096ad34b3d0f5ec.wav
425
527
before
340
540
after
340
540
(128, 200)
temp_train_labels.shape is: (200,)
(4845, 128, 200)
(4845, 200)
mixture_devtrain_gunshot_4845_e4da808fa8ce325b23d2202521444925.wav
604
626
before
510
710
after
510
710
(128, 200)
temp_train_labels.shape is: (200,)
(4846, 128, 200)
(4846, 200)
mixture_devtrain_gunshot_4846_f6b331526af79931ac2ddd08424a7001.wav
358
451
befo

(4887, 128, 200)
(4887, 200)
mixture_devtrain_gunshot_4887_d6efe16acaa62793d92b47ce69939be3.wav
119
134
before
53
253
after
53
253
(128, 200)
temp_train_labels.shape is: (200,)
(4888, 128, 200)
(4888, 200)
mixture_devtrain_gunshot_4888_a1038275fcacf346a055da060a0baf92.wav
951
1051
before
903
1103
after
903
1103
(128, 200)
temp_train_labels.shape is: (200,)
(4889, 128, 200)
(4889, 200)
mixture_devtrain_gunshot_4889_72363ecf2c0e64f03b73261d286bf997.wav
975
1040
before
893
1093
after
893
1093
(128, 200)
temp_train_labels.shape is: (200,)
(4890, 128, 200)
(4890, 200)
mixture_devtrain_gunshot_4890_7543174a2efbf8efd1c24cc13676dffe.wav
425
527
before
413
613
after
413
613
(128, 200)
temp_train_labels.shape is: (200,)
(4891, 128, 200)
(4891, 200)
mixture_devtrain_gunshot_4891_2414e10142e93b6f70892ec7dc05c95a.wav
86
124
before
52
252
after
52
252
(128, 200)
temp_train_labels.shape is: (200,)
(4892, 128, 200)
(4892, 200)
mixture_devtrain_gunshot_4892_603b57d67c206af48b53aa4e167d58dc.wav
511
527


(4936, 128, 200)
(4936, 200)
mixture_devtrain_gunshot_4936_c69e551040b96097b75f94ecb8723c11.wav
779
897
before
702
902
after
702
902
(128, 200)
temp_train_labels.shape is: (200,)
(4937, 128, 200)
(4937, 200)
mixture_devtrain_gunshot_4937_8c3cb9f07baf75fca03017459545ff78.wav
1000
1027
before
903
1103
after
903
1103
(128, 200)
temp_train_labels.shape is: (200,)
(4938, 128, 200)
(4938, 200)
mixture_devtrain_gunshot_4938_3d92589139dbdc412c18d81df6e552f6.wav
356
417
before
265
465
after
265
465
(128, 200)
temp_train_labels.shape is: (200,)
(4939, 128, 200)
(4939, 200)
mixture_devtrain_gunshot_4939_4ea7e58563e5355c31355fe688119517.wav
1085
1208
before
1080
1280
after
1080
1280
(128, 200)
temp_train_labels.shape is: (200,)
(4940, 128, 200)
(4940, 200)
mixture_devtrain_gunshot_4940_b1a5a3df93f583eb4a91242d67bfea15.wav
1060
1160
before
992
1192
after
992
1192
(128, 200)
temp_train_labels.shape is: (200,)
(4941, 128, 200)
(4941, 200)
mixture_devtrain_gunshot_4941_b06fb873738b0b6e87245d9758c11c2c

(4983, 128, 200)
(4983, 200)
mixture_devtrain_gunshot_4983_176408a559ea735d4548f5a19cf38225.wav
1088
1141
before
990
1190
after
990
1190
(128, 200)
temp_train_labels.shape is: (200,)
(4984, 128, 200)
(4984, 200)
mixture_devtrain_gunshot_4984_e72c89c825d8c85b9cccb9c549ec3f87.wav
713
737
before
659
859
after
659
859
(128, 200)
temp_train_labels.shape is: (200,)
(4985, 128, 200)
(4985, 200)
mixture_devtrain_gunshot_4985_246d576469545ab22f3a73ab92d426f4.wav
977
1008
before
914
1114
after
914
1114
(128, 200)
temp_train_labels.shape is: (200,)
(4986, 128, 200)
(4986, 200)
mixture_devtrain_gunshot_4986_bf5733e8c404322e42ddd09d24ff7185.wav
673
732
before
589
789
after
589
789
(128, 200)
temp_train_labels.shape is: (200,)
(4987, 128, 200)
(4987, 200)
mixture_devtrain_gunshot_4987_83d2ba5311cba8a8f0d515e34859ae29.wav
1085
1208
before
1043
1243
after
1043
1243
(128, 200)
temp_train_labels.shape is: (200,)
(4988, 128, 200)
(4988, 200)
mixture_devtrain_gunshot_4988_9056da954e8cc37abc2bc4f4e7394789.

In [4]:
train_frames = np.array(d['train_frames'])
train_labels = np.array(d['train_labels'])

In [5]:
f = open('train_5000_gunshot_Prob1_44100_128_framelevel_calssifier.pkl','wb')
pickle.dump(d,f)
f.close()

In [8]:
frames_per_sample = 200

def create_melspec_val(path):
    val_frames=[]
    val_labels=[]
    with open('MyCode/data/TUT-rare-sound-events-2017-development/devtrain_3000_1/5bb7b33280ceed518212918dffa8500b/meta/event_list_devtrain_gunshot.csv', 'rt') as val_file:
        reader = list(csv.reader(val_file, delimiter='\n'))
        
        for row,i in zip(reader,range(1000)):
            fields= str(row).split('\\t')
            
            temp_val_frames =[]
            temp_val_labels = []
                    
            if(len(fields)>1):
                if(fields[1] and fields[2]):
                    file_name = fields[0].replace("[\'", "").replace("\']", "")
                    melspec = imageio.imread(path + str(i) + '.png')
                    print(file_name)
                    event_start_index = int(np.round(float(fields[1])/30 * 1305))
                    event_finish_index = int(np.round(float(fields[2])/30 * 1305))
                    event_length = event_finish_index - event_start_index 
                    print(event_start_index)
                    print(event_finish_index)
                    
                    #calculating sample start and finish indices, randomly
                    sample_start = randint(max(0,event_start_index - (200-event_length)),event_start_index)
                    sample_finish = min(1305,sample_start+frames_per_sample)
                    
                    print('before')
                    print(sample_start)
                    print(sample_finish)
                    sample_length = sample_finish - sample_start 
                    if sample_length < frames_per_sample:#if we don't have enough negative samples after event
                        sample_start = sample_start - (frames_per_sample - sample_length)
                    
                    print('after')
                    print(sample_start)
                    print(sample_finish)
                    
                    temp1 = np.array(melspec)
                    temp_val_frames = temp1[:,sample_start:sample_finish]
                    
                    print(np.array(temp_val_frames).shape)
                    for ind in range(sample_start,event_start_index):
                        temp_val_labels.append(0)
                    for ind in range(event_start_index,event_finish_index):
                        temp_val_labels.append(1)
                    for ind in range(event_finish_index,sample_finish):
                        temp_val_labels.append(0)
                    
                    print('temp_val_labels.shape is: '+str(np.array(temp_val_labels).shape))   
                    temp2 = np.array(temp_val_frames)
                    temp2 = temp2.reshape(128,200)
                    val_frames.append(temp2)
                    
                    temp3 = np.array(temp_val_labels)
                    temp3 = temp3.reshape(200)
                    val_labels.append(temp3)
                    
                    print(np.array(val_frames).shape)
                    print(np.array(val_labels).shape)
            if(i%500==0):
                print('Reading files: '+str(i))
    
    val_file.close()
    val_data = {'val_frames':val_frames,
                 'val_labels':val_labels}
    return val_data

In [9]:
val_file_path = 'val_1000_gunshot_44100_128/'
v={}
v = create_melspec_val(val_file_path)

mixture_devtrain_gunshot_000_6121d48c068771d0eb749503ab4c3847.wav
492
502
before
339
539
after
339
539
(128, 200)
temp_val_labels.shape is: (200,)
(1, 128, 200)
(1, 200)
Reading files: 0
mixture_devtrain_gunshot_001_64780d6a57101d8ee59467172e65d396.wav
795
879
before
733
933
after
733
933
(128, 200)
temp_val_labels.shape is: (200,)
(2, 128, 200)
(2, 200)
mixture_devtrain_gunshot_002_10078b72b584b117f1915206e729d7c1.wav
554
671
before
492
692
after
492
692
(128, 200)
temp_val_labels.shape is: (200,)
(3, 128, 200)
(3, 200)
mixture_devtrain_gunshot_009_1bc1ce059ee64fe0a33b390548379c65.wav
1173
1272
before
1161
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(4, 128, 200)
(4, 200)
mixture_devtrain_gunshot_011_6b2f1b3bdfcccc5c9aaa601dcff2cca5.wav
868
929
before
858
1058
after
858
1058
(128, 200)
temp_val_labels.shape is: (200,)
(5, 128, 200)
(5, 200)
mixture_devtrain_gunshot_012_5a284d80ef86fd29adaf061fbd693a20.wav
365
455
before
273
473
after
273
473
(128, 200)
temp_val_la

mixture_devtrain_gunshot_158_7937099113c75abfa7b1be0d016b5640.wav
872
905
before
758
958
after
758
958
(128, 200)
temp_val_labels.shape is: (200,)
(84, 128, 200)
(84, 200)
mixture_devtrain_gunshot_164_d6b893c6af40e8d6c600b8a937798c1f.wav
627
655
before
545
745
after
545
745
(128, 200)
temp_val_labels.shape is: (200,)
(85, 128, 200)
(85, 200)
mixture_devtrain_gunshot_177_eacc814e6343bf8a477643e3409a4ea6.wav
324
360
before
170
370
after
170
370
(128, 200)
temp_val_labels.shape is: (200,)
(86, 128, 200)
(86, 200)
mixture_devtrain_gunshot_181_0b5e87aa95de715a51db5550918e1b67.wav
1023
1035
before
838
1038
after
838
1038
(128, 200)
temp_val_labels.shape is: (200,)
(87, 128, 200)
(87, 200)
mixture_devtrain_gunshot_182_359a87b717032adc26623e6108db4510.wav
118
235
before
42
242
after
42
242
(128, 200)
temp_val_labels.shape is: (200,)
(88, 128, 200)
(88, 200)
mixture_devtrain_gunshot_186_457c982a740f4bbbc5c446a5741d1723.wav
1040
1132
before
970
1170
after
970
1170
(128, 200)
temp_val_labels.shap

mixture_devtrain_gunshot_295_d7b71c1c4cf007d69d699c1ade851155.wav
1173
1272
before
1085
1285
after
1085
1285
(128, 200)
temp_val_labels.shape is: (200,)
(144, 128, 200)
(144, 200)
mixture_devtrain_gunshot_296_40d58b0478911699d7769853cb671093.wav
83
141
before
13
213
after
13
213
(128, 200)
temp_val_labels.shape is: (200,)
(145, 128, 200)
(145, 200)
mixture_devtrain_gunshot_298_6abd49c5514d64759e86860597987209.wav
315
424
before
309
509
after
309
509
(128, 200)
temp_val_labels.shape is: (200,)
(146, 128, 200)
(146, 200)
mixture_devtrain_gunshot_300_58272a83e700c6f8a06913f33f9b1ac3.wav
750
852
before
670
870
after
670
870
(128, 200)
temp_val_labels.shape is: (200,)
(147, 128, 200)
(147, 200)
mixture_devtrain_gunshot_302_c5bb932588ec2b8b4234d94f2ec8affe.wav
483
537
before
472
672
after
472
672
(128, 200)
temp_val_labels.shape is: (200,)
(148, 128, 200)
(148, 200)
mixture_devtrain_gunshot_303_46055533e4269a8045e99a12e8d6d020.wav
1092
1115
before
1062
1262
after
1062
1262
(128, 200)
temp_va

mixture_devtrain_gunshot_437_9fbda4d4091710600708d090e72e6fa0.wav
608
639
before
528
728
after
528
728
(128, 200)
temp_val_labels.shape is: (200,)
(217, 128, 200)
(217, 200)
mixture_devtrain_gunshot_441_93e72f52fe0acfedc40d89a78d38c50b.wav
197
231
before
168
368
after
168
368
(128, 200)
temp_val_labels.shape is: (200,)
(218, 128, 200)
(218, 200)
mixture_devtrain_gunshot_442_78bacb93508d8d452959274c9d37a1b0.wav
796
851
before
739
939
after
739
939
(128, 200)
temp_val_labels.shape is: (200,)
(219, 128, 200)
(219, 200)
mixture_devtrain_gunshot_445_cd92ca51b83f7e7cda9b696818490028.wav
68
99
before
55
255
after
55
255
(128, 200)
temp_val_labels.shape is: (200,)
(220, 128, 200)
(220, 200)
mixture_devtrain_gunshot_446_14cec6d331ef00fd3816dffc97bf28b3.wav
262
282
before
124
324
after
124
324
(128, 200)
temp_val_labels.shape is: (200,)
(221, 128, 200)
(221, 200)
mixture_devtrain_gunshot_447_d37c40d082ec9db1adc4588720f06b95.wav
20
115
before
5
205
after
5
205
(128, 200)
temp_val_labels.shape is:

mixture_devtrain_gunshot_558_13ff535fbcb43a97db3ec4cfd46c4a6e.wav
106
144
before
49
249
after
49
249
(128, 200)
temp_val_labels.shape is: (200,)
(282, 128, 200)
(282, 200)
mixture_devtrain_gunshot_562_404a49aa7d2a00e9cfd8e9bb1f6314a9.wav
691
772
before
597
797
after
597
797
(128, 200)
temp_val_labels.shape is: (200,)
(283, 128, 200)
(283, 200)
mixture_devtrain_gunshot_565_964536de6c49e33a7e42049191df9d7a.wav
1092
1115
before
980
1180
after
980
1180
(128, 200)
temp_val_labels.shape is: (200,)
(284, 128, 200)
(284, 200)
mixture_devtrain_gunshot_567_87262d07cf8e318ea2858725741aaf6d.wav
600
619
before
576
776
after
576
776
(128, 200)
temp_val_labels.shape is: (200,)
(285, 128, 200)
(285, 200)
mixture_devtrain_gunshot_571_aece8d17a874b2f3760eac094c2892fe.wav
1250
1267
before
1229
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(286, 128, 200)
(286, 200)
mixture_devtrain_gunshot_574_420f7b0e35afebf91437005f4c1b7ca2.wav
197
231
before
52
252
after
52
252
(128, 200)
temp_val_l

(329, 128, 200)
(329, 200)
mixture_devtrain_gunshot_660_7968e2833ef2c35299a3b14fd154279d.wav
918
936
before
870
1070
after
870
1070
(128, 200)
temp_val_labels.shape is: (200,)
(330, 128, 200)
(330, 200)
mixture_devtrain_gunshot_661_88ee28aea8a1b44332362641f5e86ef6.wav
821
944
before
752
952
after
752
952
(128, 200)
temp_val_labels.shape is: (200,)
(331, 128, 200)
(331, 200)
mixture_devtrain_gunshot_662_63bed5821ac0dc881e924423c4fb2bd5.wav
38
72
before
27
227
after
27
227
(128, 200)
temp_val_labels.shape is: (200,)
(332, 128, 200)
(332, 200)
mixture_devtrain_gunshot_664_ba0d71135e9c16517bd8f00bc596ab0b.wav
945
990
before
827
1027
after
827
1027
(128, 200)
temp_val_labels.shape is: (200,)
(333, 128, 200)
(333, 200)
mixture_devtrain_gunshot_665_d3c66c202eeb8d27488b212217bf0f8b.wav
1236
1255
before
1087
1287
after
1087
1287
(128, 200)
temp_val_labels.shape is: (200,)
(334, 128, 200)
(334, 200)
mixture_devtrain_gunshot_668_01209825ddd8d422c41c899f3a67d6db.wav
744
761
before
739
939
after
73

mixture_devtrain_gunshot_763_1e74b9ab07bc6365dcb1c4767bd14ce9.wav
164
261
before
63
263
after
63
263
(128, 200)
temp_val_labels.shape is: (200,)
(387, 128, 200)
(387, 200)
mixture_devtrain_gunshot_764_c73638385e741dcc12310fa3c08d5bb6.wav
814
887
before
773
973
after
773
973
(128, 200)
temp_val_labels.shape is: (200,)
(388, 128, 200)
(388, 200)
mixture_devtrain_gunshot_769_a176b57cab33ea052c67c30b8d145165.wav
355
485
before
313
513
after
313
513
(128, 200)
temp_val_labels.shape is: (200,)
(389, 128, 200)
(389, 200)
mixture_devtrain_gunshot_774_da6fdc33b3f7000ba5b3291a534b21e6.wav
903
1037
before
895
1095
after
895
1095
(128, 200)
temp_val_labels.shape is: (200,)
(390, 128, 200)
(390, 200)
mixture_devtrain_gunshot_775_17fafb2367ead10e32ccc53cfe7279db.wav
840
898
before
775
975
after
775
975
(128, 200)
temp_val_labels.shape is: (200,)
(391, 128, 200)
(391, 200)
mixture_devtrain_gunshot_777_7d3cbd71df13f8ae1f2741623f21a71a.wav
364
400
before
274
474
after
274
474
(128, 200)
temp_val_labels

(439, 128, 200)
(439, 200)
mixture_devtrain_gunshot_875_674ee1e12a2d06d07d232c15a36d6569.wav
604
633
before
444
644
after
444
644
(128, 200)
temp_val_labels.shape is: (200,)
(440, 128, 200)
(440, 200)
mixture_devtrain_gunshot_878_809367d8b9e2139d73787b887d92ce92.wav
1056
1081
before
891
1091
after
891
1091
(128, 200)
temp_val_labels.shape is: (200,)
(441, 128, 200)
(441, 200)
mixture_devtrain_gunshot_884_58530a13d42072fbdf48c285e10f2a09.wav
483
537
before
355
555
after
355
555
(128, 200)
temp_val_labels.shape is: (200,)
(442, 128, 200)
(442, 200)
mixture_devtrain_gunshot_888_8884ed123d12f3b68e2bd5635d73e82b.wav
118
235
before
70
270
after
70
270
(128, 200)
temp_val_labels.shape is: (200,)
(443, 128, 200)
(443, 200)
mixture_devtrain_gunshot_889_f869fd36fdaffb50ffb47050a2c73e79.wav
1128
1177
before
1068
1268
after
1068
1268
(128, 200)
temp_val_labels.shape is: (200,)
(444, 128, 200)
(444, 200)
mixture_devtrain_gunshot_891_2b06006cdb14171947af8eddb5965661.wav
1129
1154
before
1034
1234
af

(490, 200)
mixture_devtrain_gunshot_982_56fdabaef1620da31b49b77610589284.wav
207
270
before
72
272
after
72
272
(128, 200)
temp_val_labels.shape is: (200,)
(491, 128, 200)
(491, 200)
mixture_devtrain_gunshot_983_4f734fe04cdc4a4971d3e369159ff3e6.wav
1250
1267
before
1174
1305
after
1105
1305
(128, 200)
temp_val_labels.shape is: (200,)
(492, 128, 200)
(492, 200)
mixture_devtrain_gunshot_984_8a9f7f9fbc0a35e5d47afa72d019b210.wav
197
231
before
169
369
after
169
369
(128, 200)
temp_val_labels.shape is: (200,)
(493, 128, 200)
(493, 200)
mixture_devtrain_gunshot_985_ff335418a4aea26dd9738d4a93a627d3.wav
625
726
before
543
743
after
543
743
(128, 200)
temp_val_labels.shape is: (200,)
(494, 128, 200)
(494, 200)
mixture_devtrain_gunshot_987_e28672ee917ec7933f015ece842f1bee.wav
1129
1154
before
1081
1281
after
1081
1281
(128, 200)
temp_val_labels.shape is: (200,)
(495, 128, 200)
(495, 200)
mixture_devtrain_gunshot_990_1d4b07206379f4217984c2c1194bd173.wav
306
402
before
268
468
after
268
468
(128, 

In [10]:
val_frames = np.array(v['val_frames'])
val_labels = np.array(v['val_labels'])

In [11]:
f = open('val_1000_gunshot_Prob1_44100_128__framelevel_calssifier.pkl','wb')
pickle.dump(v,f)
f.close()

In [174]:
def get_model():
    
    
    inp = Input(shape=(128,200,1))
    x = Conv2D(16, (5,5), activation='relu', padding="valid")(inp)
    x = MaxPooling2D((5,5))(x)
    x = Dropout(rate=0.3)(x)
    
    
    
    x = Conv2D(64, (3,3), activation='relu', padding="valid")(x)
    x = MaxPooling2D((3,3))(x)
    x = Dropout(rate=0.3)(x)
    
    x = Conv2D(128, (3,3), activation='relu', padding="valid")(x)
    x = MaxPooling2D((3,3))(x)
    x = Dropout(rate=0.3)(x)
    
    x = Flatten()(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(200, activation='sigmoid')(x)
    
    
    
    model = Model(inputs=inp, outputs=out)
    opt = optimizers.Adam(lr=0.001)

    model.compile(optimizer=opt, loss=losses.binary_crossentropy, metrics=['acc'])
    return model


In [175]:
my_model = get_model()

In [176]:
my_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 128, 200, 1)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 124, 196, 16)      416       
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 24, 39, 16)        0         
_________________________________________________________________
dropout_58 (Dropout)         (None, 24, 39, 16)        0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 22, 37, 64)        9280      
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 7, 12, 64)         0         
_________________________________________________________________
dropout_59 (Dropout)         (None, 7, 12, 64)         0         
__________

In [177]:
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=20, mode='auto')

In [180]:
train_frames = np.array(train_frames).reshape(5000,128,200,1)
val_frames = np.array(val_frames).reshape(500,128,200,1)
train_labels = np.array(train_labels).reshape(5000,200)
val_labels = np.array(val_labels).reshape(500,200)

In [181]:
my_model.fit(x=train_frames,y=train_labels,batch_size=200,epochs=150, 
             validation_data=(val_frames, val_labels), callbacks=[earlystop])

Train on 5000 samples, validate on 500 samples
Epoch 1/150
5000/5000 [==============================] - 53s 11ms/step - loss: 1.4878 - acc: 0.5745 - val_loss: 0.6890 - val_acc: 0.6793
Epoch 2/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6871 - acc: 0.6748 - val_loss: 0.6846 - val_acc: 0.6923
Epoch 3/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6827 - acc: 0.6819 - val_loss: 0.6804 - val_acc: 0.6933
Epoch 4/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6785 - acc: 0.6807 - val_loss: 0.6763 - val_acc: 0.6937
Epoch 5/150
5000/5000 [==============================] - 51s 10ms/step - loss: 0.6745 - acc: 0.6761 - val_loss: 0.6723 - val_acc: 0.6935
Epoch 6/150
5000/5000 [==============================] - 50s 10ms/step - loss: 0.6699 - acc: 0.6669 - val_loss: 0.6683 - val_acc: 0.6937
Epoch 7/150
5000/5000 [==============================] - 50s 10ms/step - loss: 0.6617 - acc: 0.6468 - val_loss: 0.6651 - val_acc: 0

KeyboardInterrupt: 